Ideas from Garrison:
Heuristics:
1) Avoiding non-common codons in your species
2) Add AT to GC rich regions
3) Avoid stem loops for mRNA
	1) Avoid palindromes with gaps of more than 4 base pairs
	2) Consider 4-6 gaps and high match scores
4) Improve sequence upstream of ATG (start methionine)
	1) List of sequences to specifically avoid before start codon
	2) Bias upstream towards thymine, cytosine, for 15 BP upstream of any methionine

General flow:
Provide each heuristic as a function. They will all likely be manipulating codons, so each should have as input an alternate codon list with "scores" for each codon. The scores will be used to determine which codon to use.
The scores will most likely refer to the frequency of the codon in the organism of interest, so the heuristics will be biased towards using more frequent codons.

# 1) Avoiding non-common codons in your species
We'll need:
- a table of codons for each amino acid
- a table of codon frequencies for organism of interest

What's our threshhold for "common" vs "non-common"? Should we just avoid below some number, keeping the rest the same? Or keep a "frequent enough" dict we can use to consider while we're optimizing for AT content later?

(from CoPilot:)
We'll use the codon table from [here](https://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N) and the codon frequency table from [here](https://www.genscript.com/tools/codon-frequency-table).

We'll also need a sequence to optimize. We'll use the sequence of the [SARS-CoV-2 spike protein](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2?report=fasta&log$=seqview&format=text).

# 2) Add AT to GC rich regions
Make a rolling window (how long?) calculating GC content (what threshhold is bad?).
If below, we'll switch out some codons for AT rich ones. (Remember to not use infrequent ones!)

# 3) Avoid stem loops for mRNA
We'll have to go through our DNA sequence so far and calculate stem loops. Biopython might have this functionality. If not, we'll have to write it ourselves.
ChatGPT says I can use ViennaRNA.

# 4) Improve sequence upstream of ATG (start methionine)
Garrison will give us a specific set of sequences to avoid
Bias upstream sequences STRONGLY away from banned sequences for 15BP upstream of any methionine, and towards cytosine, thymine.

In [1]:
import urllib.request as urlreq
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, Input, Output, State, ALL, callback, callback_context
import dash_bio as dashbio
from dash_bio.utils import protein_reader
from dash.exceptions import PreventUpdate

import Bio
from Bio import SeqIO

import json

In [2]:
class SequenceFilter():
    def __init__(self, sequence, filter_type):
        self.sequence = sequence
        self.filter_type = filter_type
        self.filtered_sequence = self.filter_sequence()
    
    def filter_sequence(self):
        if self.filter_type == 'codon':
            return self.filter_codon()
        elif self.filter_type == 'aa':
            return self.filter_aa()
        else:
            raise ValueError('filter_type must be codon or aa')


In [3]:
from Bio.Data import CodonTable

codon_to_AA = CodonTable.unambiguous_dna_by_name['Standard'].forward_table
#These are the codon frequencies for E.Coli for each codon
AA_freq_dict = {'F': {'TTT': 0.58, 'TTC': 0.42}, 'L': {'TTA': 0.14, 'TTG': 0.13, 'CTT': 0.12, 'CTC': 0.1, 'CTA': 0.04, 'CTG': 0.47}, 'Y': {'TAT': 0.59, 'TAC': 0.41}, '*': {'TAA': 0.61, 'TAG': 0.09, 'TGA': 0.3}, 'H': {'CAT': 0.57, 'CAC': 0.43}, 'Q': {'CAA': 0.34, 'CAG': 0.66}, 'I': {'ATT': 0.49, 'ATC': 0.39, 'ATA': 0.11}, 'M': {'ATG': 1.0}, 'N': {'AAT': 0.49, 'AAC': 0.51}, 'K': {'AAA': 0.74, 'AAG': 0.26}, 'V': {'GTT': 0.28, 'GTC': 0.2, 'GTA': 0.17, 'GTG': 0.35}, 'D': {'GAT': 0.63, 'GAC': 0.37}, 'E': {'GAA': 0.68, 'GAG': 0.32}, 'S': {'TCT': 0.17, 'TCC': 0.15, 'TCA': 0.14, 'TCG': 0.14, 'AGT': 0.16, 'AGC': 0.25}, 'C': {'TGT': 0.46, 'TGC': 0.54}, 'W': {'TGG': 1.0}, 'P': {'CCT': 0.18, 'CCC': 0.13, 'CCA': 0.2, 'CCG': 0.49}, 'R': {'CGT': 0.36, 'CGC': 0.36, 'CGA': 0.07, 'CGG': 0.11, 'AGA': 0.07, 'AGG': 0.04}, 'T': {'ACT': 0.19, 'ACC': 0.4, 'ACA': 0.17, 'ACG': 0.25}, 'A': {'GCT': 0.18, 'GCC': 0.26, 'GCA': 0.23, 'GCG': 0.33}, 'G': {'GGT': 0.35, 'GGC': 0.37, 'GGA': 0.13, 'GGG': 0.15}}

In [10]:
#each filter needs:
# to hold constant an AA sequence
# update nucleotide sequence as provided
# calculate scores per codon
# return scores
# return color mapping based on scores
# return suggestions based on scores
'''
workflow
initialize with sequence
generate persistent AA sequence

process() to calculate scores
get_scores() to return scores
get_color_mapping() to return color mapping
get_suggestions() to return suggestions

'''
from Bio.Data import CodonTable

codon_to_AA = CodonTable.unambiguous_dna_by_name['Standard'].forward_table

class FrequencyFilter():
    def __init__(self, sequence):
        self.sequence = sequence

    def process(self, AA_freq_dict):
        #given self.sequence,
        #returns frequency of each codon in the sequence
        #as a list of floats. The list will contain a float for every nucleotide in the sequence, representing frequency.

        #split string into codons:
        codon_list = [self.sequence[i:i+3] for i in range(0, len(self.sequence), 3)]

        #get AA for each codon in input:
        AA_list = []
        for codon in codon_list:
            if codon not in codon_to_AA.keys():
                AA_list.append('X')
            else:
                AA_list.append(codon_to_AA[codon])

        #get the frequency for each amino acid in the sequence
        frequency_list = []
        for i in range(len(AA_list)):
            AA = AA_list[i]
            if AA == 'X':
                frequency_list += [0]*3
            else:
                frequency_list += [AA_freq_dict[AA][codon_list[i]]]*3

        #for each codon, suggest the most frequent codon for that AA
        suggestion_list = []
        for AA in AA_list:
            if AA == 'X':
                suggestion_list.append([])
            else:
                #most frequent codon for that AA
                suggestion_list.append(sorted(AA_freq_dict[AA].keys(), key=AA_freq_dict[AA].get, reverse=True))
        
        return frequency_list, suggestion_list


CGCTGGCGGCGTGCTAATACATGCCCGTCGAGCNGGATCGATGGGAGCTTGCTCCCTGCAGATCAGCGGCGGACGGGTGAGTAACACGTGGGTAACCTGCCTGTAAGACTGGGATAACTCCGGGAAACCGGGGCTAATACCGGATAATTTAGTTCCTCGCATGAGGAACTGTTGAAAGGTGGCTTC
---------------------------------------------------------------------------
JSONDecodeError                           Traceback (most recent call last)
File ~/miniconda3/envs/codon_opt/lib/python3.8/json/__init__.py:357, in loads(
    s='',
    cls=None,
    object_hook=None,
    parse_float=None,
    parse_int=None,
    parse_constant=None,
    object_pairs_hook=None,
    **kw={}
)
    352     del kw['encoding']
    354 if (cls is None and object_hook is None and
    355         parse_int is None and parse_float is None and
    356         parse_constant is None and object_pairs_hook is None and not kw):
--> 357     return _default_decoder.decode(s)
        s = ''
        _default_decoder = <json.decoder.JSONDecoder object at 0x7fbd61cca160>
    358 if cls is None:
    359     cls = JSONDecoder

File 

In [5]:
def create_sequence_viewer_app(starting_sequence, filter_list):
    """
    Creates a dash app that displays a number of stacked sequence viewers, given the number of filters.
    Returns the app object and a list of IDs for the sequence viewers.
    """

    #app = Dash(__name__)
    app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

    ### Sidebar Layout ###
    SIDEBAR_STYLE = {
        "position": "fixed",
        "top": 0,
        "right": 0,
        "bottom": 0,
        "width": "16rem",
        "padding": "2rem 1rem",
        "background-color": "#f8f9fa",
    }

    # the styles for the main content position it to the right of the sidebar and
    # add some padding.
    CONTENT_STYLE = {
        "margin-left": "18rem",
        "margin-right": "2rem",
        "padding": "2rem 1rem",
        "background-color": "white",
    }

    ### Submission Box Layout ###
    SUBMISSION_BOX_STYLE = {
        "position": "fixed",
        "top": 0,
        "bottom": 0,
        "left": 0,
        "width": "16rem",
        "padding": "2rem 1rem",
        "background-color": "#f8f9fa",
    }

    ### Submission Box ###
    submission_box = html.Div(
        [
            html.H3("Submit", className="display-4"),
            html.Hr(),
            html.Div(id="submission-box-content", children=[
                html.P(
                "Submit your sequence here.", className="lead"
                ),
                dcc.Textarea(
                    id='submission-box',
                    value=starting_sequence,
                    style={'width': '100%', 'height': 300},
                ),
                html.Button('Submit', id='submit-button', className='btn btn-primary')
                
            ]),
        ],
        style=SUBMISSION_BOX_STYLE,
    )

    sidebar = html.Div(
        [
            html.H3("Sidebar", className="display-4"),
            html.Hr(),
            html.Div(id="sidebar-content", children=[
                html.P(
                "Suggestions will be shown here.", className="lead"
                )
            ]),
        ],
        style=SIDEBAR_STYLE,
    )

    
    content = html.Div(
        [ dashbio.SequenceViewer(
            id='default-sequence-viewer-{}'.format(i),
            sequence='AAA',
            toolbar=False,
            badge=False,
            charsPerLine=90,
            search=False,
        ) for i in range(len(filter_list)) ]+
        [html.Div(id='output')],
        style=CONTENT_STYLE
    )

    app.layout = html.Div(
            [dcc.Location(id="url"), sidebar, content, submission_box,
            # store the working sequence
            dcc.Store(id='sequence'),
            # store the previous sequence 
            dcc.Store(id='previous_sequence'),
            dcc.Store(id='freq_list_per_filter'),
            dcc.Store(id='suggestion_list_per_filter'),
            dcc.Store(id='clicked_index'),
            dcc.Store(id='clicked_filter'),
            ]
        )

    
    '''
    Workflow: User submits sequence in submission box.
    This updates the current_sequence store.
    This triggers the callback to update the filters + suggestions
    This triggers the callback to update the sequence viewers + sidebar

    On clicking a suggestion, the sequence is updated again. Will this work?
    '''

    @app.callback(
        [Output("freq_list_per_filter", "data"), Output("suggestion_list_per_filter", "data")],
        [Input("sequence", "data")],
    )
    def run_filters(seq):
        #filter_list is static and passed in to wrapper function
        
        #get output of each filter:
        freq_list_per_filter = []
        suggestion_list_per_filter = []
        for filter in filter_list:
            filter.sequence = seq
            freq_list, suggestion_list = filter.process(AA_freq_dict)
            freq_list_per_filter.append(freq_list)
            suggestion_list_per_filter.append(suggestion_list)
        return freq_list_per_filter, suggestion_list_per_filter
    
    @app.callback(
        [Output('default-sequence-viewer-{}'.format(i), 'coverage') for i in range(len(filter_list))]+
        [Output('default-sequence-viewer-{}'.format(i), 'sequence') for i in range(len(filter_list))]+
        [Output('sidebar-content', 'children')],
        [[Input('default-sequence-viewer-{}'.format(i), 'mouseSelection') for i in range(len(filter_list))],
        Input('freq_list_per_filter', 'data'),
        Input('suggestion_list_per_filter', 'data'),
        State('sequence', 'data')]
    )
    def update_highlighting_and_suggestions(mouseSelections, freq_list_per_filter, suggestion_list_per_filter, current_sequence):
        
        #get index
        index = None
        chosen_filter = None
        for i, mouseSelection in enumerate(mouseSelections):
            if mouseSelection is not None:
                index = mouseSelection['start']-1 #not 0 indexed lol
                chosen_filter = i
                break
        
        print("updating highlighting")
        coverages = update_highlighting(index, freq_list_per_filter)

        print("updating suggestions")
        sidebar_children = update_suggestions(index, chosen_filter, suggestion_list_per_filter)

        print("updating sequence")
        sequences = [current_sequence]*len(filter_list)
        return coverages + sequences + sidebar_children


    def update_highlighting(index, freq_list_per_filter):
        
        coverages = []
        if freq_list_per_filter is not None:
            for freq_list in freq_list_per_filter:
                
                #convert to colors
                color_list = []
                for i in range(len(freq_list)):
                    if freq_list[i] > 0.75:
                        color_list.append('green')
                    elif freq_list[i] > 0.5:
                        color_list.append('yellow')
                    elif freq_list[i] > 0.25:
                        color_list.append('orange')
                    else:
                        color_list.append('red')
                
                #make coverage
                coverage = [
                    {'start': i, 'end': i+1, 'bgcolor': color_list[i]} for i in range(len(freq_list))
                ]

                if index is not None:
                    #get codon containing selection:
                    codon = index//3
                    #get 3 nucleotides in codon:
                    indices = [codon*3, codon*3+1, codon*3+2]
                    #set all nucleotides in codon to black:
                    for index in indices:
                        coverage[index]['underscore'] = True
                coverages.append(coverage)
        else:
            coverages = [[]]*len(filter_list)
        return coverages
    
    def update_suggestions(index, chosen_filter, suggestion_list_per_filter):

        if index is None:
            return [html.P(
                "Suggestions will be shown here.", className="lead"
            )]
        else:
            suggestions = suggestion_list_per_filter[chosen_filter][index//3]
            buttons = [html.Button(suggestion, id={'type': 'suggestion-button', 'index': i}, className='btn btn-primary') for i, suggestion in enumerate(suggestions)]
            return [buttons]
    
    @app.callback(
        [Output("sequence", "data")],
        [Input({'type': 'suggestion-button', 'index': ALL}, 'n_clicks')],
        [Input("submit-button", 'n_clicks')],
        [State("submission-box", "value")],
        [State({'type': 'suggestion-button', 'index': ALL}, 'id')],
        [State('sequence', 'data')],
        [State('suggestion_list_per_filter', 'data')],
        [[State('default-sequence-viewer-{}'.format(i), 'mouseSelection') for i in range(len(filter_list))]]
    )
    def handle_button_clicks(n_clicks_list,  submit_button_nclicks, submitted_sequence, id_list,  current_sequence, suggestion_list_per_filter, mouseSelections):

        print(submitted_sequence)
        
        ctx = callback_context
        triggered_id = ctx.triggered_id
        if triggered_id == 'submit-button':
            print('submit button clicked')
            print(submitted_sequence)
            return [submitted_sequence]

        #otherwise, it's the suggestion buttons
        
        # if not ctx.triggered:
        #     button_id = 'No clicks yet'
        #     raise PreventUpdate
        # else:
        #     button_id = json.loads(ctx.triggered[0]['prop_id'].split('.')[0])['index']
        
        button_id = json.loads(ctx.triggered[0]['prop_id'].split('.')[0])['index']

        #get index
        index = None
        chosen_filter = None
        for i, mouseSelection in enumerate(mouseSelections):
            if mouseSelection is not None:
                index = mouseSelection['start']-1 #not 0 indexed lol
                chosen_filter = i
                break

        if n_clicks_list[button_id] is not None:
            #return "Button {} was clicked {} times".format(button_id, n_clicks_list[button_id])
            #Now we have a way of getting a button click!
            #N.B. need to pass through WHICH filter we're talking about.
            print('mouseSelections', chosen_filter)
            print('index', index)
            print('suggestion_list_per_filter', suggestion_list_per_filter)
            codon_index = index//3
            suggestion_list = suggestion_list_per_filter[chosen_filter]
            print('suggestion_list', suggestion_list)
            #get the suggestion
            suggestion = suggestion_list[codon_index][button_id]
            print('suggestion', suggestion)
            #set the first codon to the suggestion
            print(suggestion)
            print(current_sequence)
            
            #insert suggestion in the proper codon:
            codon_start = codon_index*3
            codon_end = codon_start+3
            new_sequence = current_sequence[:codon_start] + suggestion + current_sequence[codon_end:]

            return [new_sequence]
        #else it hasn't been pushed yet
        
        print('no button clicked')
        raise PreventUpdate
            


    return app, ['default-sequence-viewer-{}'.format(i) for i in range(len(filter_list))]
    

In [6]:
def runner(fasta):

    # Read in the fasta file
    with open('AM231703.1.fasta', 'r') as f:
        fasta_str = f.read()

    seq = protein_reader.read_fasta(datapath_or_datastring=fasta_str, is_datafile=False)[0]['sequence']

    # detect open reading frames to preserve:
    # assume ORF 0
    #ORF = 0

    freq_filter = FrequencyFilter(seq)

    app, ids = create_sequence_viewer_app(seq, [freq_filter, freq_filter, freq_filter])

    app.run(debug=True)

runner('AM231703.1.fasta')


CGCTGGCGGCGTGCTAATACATGCCCGTCGAGCNGGATCGATGGGAGCTTGCTCCCTGCAGATCAGCGGCGGACGGGTGAGTAACACGTGGGTAACCTGCCTGTAAGACTGGGATAACTCCGGGAAACCGGGGCTAATACCGGATAATTTAGTTCCTCGCATGAGGAACTGTTGAAAGGTGGCTTC
---------------------------------------------------------------------------
JSONDecodeError                           Traceback (most recent call last)
File ~/miniconda3/envs/codon_opt/lib/python3.8/json/__init__.py:357, in loads(
    s='',
    cls=None,
    object_hook=None,
    parse_float=None,
    parse_int=None,
    parse_constant=None,
    object_pairs_hook=None,
    **kw={}
)
    352     del kw['encoding']
    354 if (cls is None and object_hook is None and
    355         parse_int is None and parse_float is None and
    356         parse_constant is None and object_pairs_hook is None and not kw):
--> 357     return _default_decoder.decode(s)
        s = ''
        _default_decoder = <json.decoder.JSONDecoder object at 0x7fbd61cca160>
    358 if cls is None:
    359     cls = JSONDecoder

File 

In [8]:
'''
workflow
initialize with sequence
generate persistent AA sequence

process() to calculate scores
get_annotations() to return scores
score_to_color() to return color mapping
apply_suggestion() to apply a suggestion to the sequence and return the new sequence

'''

#Todo: would be good to have there be either "codon" or "nucleotide" filters.
#Or better yet, have the suggestions contain a [start, end] index for the region to change (nucleotide, codon, or larger region).
class SequenceFilter():
    def __init__(self, sequence, title):
        self.sequence = sequence
        self.title = title
        self.AA_sequence = self.generate_AA_sequence(self.sequence) #persistent
        self.annotations = None

        self.process()

    def process(self):
        '''
        For a stored sequence, this updates the scores and suggestions based on the implemented filter.
        Input: 
            None, but uses self.sequence
            self.sequence: a string representing the nucleotide sequence
        Output:
            None, but updates self.annotations
            self.annotations:
                This breaks the sequence into regions, and gives the scores and suggestions for each region.
                A list of dicts. Each dict is of the form:
                    { 'start': start_index,
                      'end': end_index,
                      'score': score,
                      'suggestions': [ (suggestion_1, score_1), (suggestion_2, score_2), ... ]}
        '''
        raise NotImplementedError
    
    def score_to_color(self, score):
        '''
        Given a score, returns a color.
        Input:
            score: a float between 0 and 1
        Output:
            color: a string representing a color to be used by Dash
        '''
        raise NotImplementedError

    def get_sequence(self):
        return self.sequence

    def get_AA_sequence(self):
        return self.AA_sequence

    def update_sequence(self, new_sequence, force=False):
        if not force:
            #confirm that the new sequence has same AA as old sequence:
            new_AA_sequence = self.generate_AA_sequence(new_sequence)
            if new_AA_sequence != self.AA_sequence:
                raise ValueError('new sequence must have same AA sequence as old sequence.\nold AA sequence: {}\nnew AA sequence: {}\n To override, set force=True'.format(self.AA_sequence, new_AA_sequence))
        else:
            #if we're forcing it, set the new AA sequence regardless
            self.AA_sequence = self.generate_AA_sequence(new_sequence)

        #update the sequence
        self.sequence = new_sequence
    
    def generate_AA_sequence(self, s):
        #given sequence s,
        #returns the AA sequence
        #as a string
        return ''.join([codon_to_AA[s[i:i+3]] for i in range(0, len(s), 3)])
    
    def get_annotations(self):
        '''
        Returns the annotations for the current sequence.
        '''
        return self.annotations
    
    def apply_suggestion(self, start, end, suggestion, force=False):
        '''
        Given a suggestion, applies it to the sequence and returns the new sequence.
        Input:
            start: (int) start index of the region to change
            end:   (int) end index of the region to change
            suggestion: (string) suggestion
            force: (bool) if True, will apply the suggestion even if it changes the AA sequence
        Output:
            None, but updates self.sequence
            self.sequence: (string) the new sequence
        '''
        #check that the suggestion is the right length:
        if len(suggestion) != end-start:
            raise ValueError('suggestion must be the same length as the region to change.\nstart: {}\nend: {}\nsuggestion: {}'.format(start, end, suggestion))
        
        #apply the suggestion
        new_sequence = self.sequence[:start] + suggestion + self.sequence[end:]

        #update the sequence
        self.update_sequence(new_sequence, force=force)




In [9]:
#make a codon frequency filter
class FrequencyFilter(SequenceFilter):
    def __init__(self, sequence, title='Frequency Filter'):
        super().__init__(sequence, title)
    
    def process(self):
        '''
        For a stored sequence, this updates the scores and suggestions based on the implemented filter.
        Input: 
            None, but uses self.sequence
            self.sequence: a string representing the nucleotide sequence
        Output:
            None, but updates self.annotations
            self.annotations:
                This breaks the sequence into regions, and gives the scores and suggestions for each region.
                A list of dicts. Each dict is of the form:
                    { 'start': start_index,
                      'end': end_index,
                      'score': score,
                      'suggestions': [ (suggestion_1, score_1), (suggestion_2, score_2), ... ]}
        '''

        #split string into codons:
        codon_list = [self.sequence[i:i+3] for i in range(0, len(self.sequence), 3)]

        #get AA for each codon in input:
        AA_list = []
        for codon in codon_list:
            if codon not in codon_to_AA.keys():
                AA_list.append('X')
            else:
                AA_list.append(codon_to_AA[codon])

        #get the frequency for each amino acid in the sequence
        frequency_list = []
        for i in range(len(AA_list)):
            AA = AA_list[i]
            if AA == 'X':
                frequency_list += [0]*3
            else:
                frequency_list += [AA_freq_dict[AA][codon_list[i]]]*3

        #for each codon, suggest the most frequent codon for that AA
        suggestion_list = []
        for AA in AA_list:
            if AA == 'X':
                suggestion_list.append([])
            else:
                #most frequent codon for that AA
                suggestion_list.append(sorted(AA_freq_dict[AA].keys(), key=AA_freq_dict[AA].get, reverse=True))

        #make the annotations
        self.annotations = []
        for i in range(len(codon_list)):
            self.annotations.append({
                'start': i*3,
                'end': i*3+3,
                'score': frequency_list[i],
                'suggestions': suggestion_list[i]
            })

    def score_to_color(self, score):
        '''
        Given a score, returns a color.
        Input:
            score: a float between 0 and 1
        Output:
            color: a string representing a color to be used by Dash
        '''
        if score > 0.75:
            return 'green'
        elif score > 0.5:
            return 'yellow'
        elif score > 0.25:
            return 'orange'
        else:
            return 'red'

In [ ]:
from Bio.Data import CodonTable
codon_to_AA = CodonTable.unambiguous_dna_by_name['Standard'].forward_table


In [5]:
s = "GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA".replace('U', 'T')
s

'GAGTAGTGGAACCAGGCTATGTTTGTGACTCGCAGACTAACA'

In [2]:
s

'GAGTAGTGGAACCAGGCTATGTTTGTGACTCGCAGACTAACA'

In [3]:
import RNA

# The RNA sequence
seq = "GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA"
ss  = "..( ((( (.. ... ... ))) ))( ((( ((( ... ))) ))) ).. ..."

# create a new model details structure
md = RNA.md()
# change temperature and dangle model
md.temperature = 20.0 # 20 Deg Celcius

# compute minimum free energy (MFE) and corresponding structure
fc = RNA.fold_compound(seq, md)
(ss, mfe) = fc.mfe()

# print output
print("{}\n{} [ {:6.2f} ]".format(seq, ss, mfe)) #kcal/mol

GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA
..(((((........)))))(((((((...)))))))..... [  -8.80 ]


In [ ]:
..(((((........)))))(((((((...)))))))..... [ -13.60 ]
..(((((........)))))(((((((...)))))))..... [  -8.80 ]

In [16]:
seq = "CGCTGGCGGCGTGCTAATACATGCCCGTCGAGCNGGATCGATGGGAGCTTGCTCCCTGCAGATCAGCGGCGGACGGGTGAGTAACACGTGGGTAACCTGCCTGTAAGACTGGGATAACTCCGGGAAACCGGGGCTAATACCGGATAATTTAGTTCCTCGCATGAGGAACTGTTGAAAGGTGGCTTC"
ff  = "..(((((........)))))(((((((...)))))))....."
# compute minimum free energy (MFE) and corresponding structure
(ss, mfe) = RNA.fold(seq)

# print output
print("{}\n{} [ {:6.2f} ]".format(seq, ss, mfe))

CGCTGGCGGCGTGCTAATACATGCCCGTCGAGCNGGATCGATGGGAGCTTGCTCCCTGCAGATCAGCGGCGGACGGGTGAGTAACACGTGGGTAACCTGCCTGTAAGACTGGGATAACTCCGGGAAACCGGGGCTAATACCGGATAATTTAGTTCCTCGCATGAGGAACTGTTGAAAGGTGGCTTC
...((.(.(((((....(((.((((((((..((..((((...(((((....)))))....)))).))....)))))))).))).))))).).))..((((((......((((..((.((((((....)))))).))...))))......(((((((((....))))))))).....)))))).... [ -72.40 ]


In [ ]:
....(((........)))..(((((((...))))))).....
.....((((...((.((.....)).))..)))).........